<font size=5>比赛链接：[飞桨学习赛：遥感影像地块分割](https://aistudio.baidu.com/aistudio/competition/detail/63/0/introduction)
</font>


## 0、赛题介绍
* 本赛题由 2020 CCF BDCI 遥感影像地块分割 初赛赛题改编而来。遥感影像地块分割, 旨在对遥感影像进行像素级内容解析，对遥感影像中感兴趣的类别进行提取和分类，在城乡规划、防汛救灾等领域具有很高的实用价值，在工业界也受到了广泛关注。现有的遥感影像地块分割数据处理方法局限于特定的场景和特定的数据来源，且精度无法满足需求。因此在实际应用中，仍然大量依赖于人工处理，需要消耗大量的人力、物力、财力。本赛题旨在衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果，利用人工智能技术，对多来源、多场景的异构遥感影像数据进行充分挖掘，打造高效、实用的算法，提高遥感影像的分析提取能力。 赛题任务 本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

* 本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

![](https://ai-studio-static-online.cdn.bcebos.com/a829cd4f77cf4ad79a887e6b6151593f8aae2fb804364296a20ad62ace079fa7)



### 数据集特性
![](https://ai-studio-static-online.cdn.bcebos.com/26f2b1cc7b514c088e5f3bcf4718c1caedb482a89f65478cb3cdd76406b6ba73)
![](https://ai-studio-static-online.cdn.bcebos.com/36ca749a37bc477180e9be08cf5f14e8eff3792fc1704f0a9b10bf07f3405923)




## 1、环境配置

In [1]:
# 克隆paddleSeg的github仓库
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git
!pip install -r PaddleSeg/requirements.txt

正克隆到 'PaddleSeg'...
remote: Enumerating objects: 20171, done.
remote: Counting objects: 100% (5140/5140), done.
remote: Compressing objects: 100% (2425/2425), done.
remote: Total 20171 (delta 3243), reused 4335 (delta 2647), pack-reused 15031
接收对象中: 100% (20171/20171), 345.54 MiB | 6.09 MiB/s, 完成.
处理 delta 中: 100% (13103/13103), 完成.
检查连接... 完成。
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


## 2、数据准备

In [2]:
# 解压数据集
!mkdir ~/PaddleSeg/datasets
!unzip -q data/data77571/train_and_label.zip -d ~/PaddleSeg/datasets
!unzip -q data/data77571/img_test.zip -d ~/PaddleSeg/datasets

## 3、数据预处理

### （1）数据集划分模块

In [1]:
import numpy as np
import os
os.chdir('/home/aistudio/PaddleSeg/datasets/')

datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

total: 67424
img_train/T030431.jpg
lab_train/T030431.png
('img_train/T095861.jpg', 'lab_train/T095861.png')


### （2）划分数据集

In [2]:
import cv2
import numpy as np
import os
os.chdir('/home/aistudio/PaddleSeg/datasets/')
# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地', '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(3407)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为验证集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('val_list.txt', 'w') as f:
    for img, lbl in valid_data:
        # 进行数据清洗，数据验证过程中，对于全为255的图像直接忽略
        clean = cv2.imread(lbl)
        if (clean == 255).all():
            continue
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))

train: 64053
valid: 3371


### （3）分析数据类别样本数量[1]

In [3]:
import cv2
import numpy as np

NUM_CLASSES = 4

area = {i : 0 for i in range(NUM_CLASSES)}
area_proportion = {i : {0 : 0, 1 : 0, 2 : 0, 3 : 0} for i in range(NUM_CLASSES)}
area[255] = 0
image_num = 0

def calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    for cls in range(num_classes):
        area[cls] += np.count_nonzero(label_image == cls)
    area[255] += np.count_nonzero(label_image == 255)

def area_calc(image, num_classes=NUM_CLASSES):
    label_image = np.array(image)
    image_area = label_image.shape[0] * label_image.shape[1]
    for cls in range(num_classes):
        proportion = np.count_nonzero(label_image == cls) / float(image_area)
        if proportion < 0.01:
            area_proportion[cls][0] += 1
        elif proportion < 0.02:
            area_proportion[cls][1] += 1
        elif proportion < 0.8:
            area_proportion[cls][2] += 1
        else:
            area_proportion[cls][3] += 1


# 统计四种类型的面积占比
train_file_dir = '/home/aistudio/PaddleSeg/datasets/train_list.txt'
val_file_dir = '/home/aistudio/PaddleSeg/datasets/val_list.txt'
with open(train_file_dir, 'r') as f:
    for line in f.readlines():
        if image_num % 5000 == 0:
            print("当前已读取"+str(image_num)+"个样本，进度为 "+str(100*image_num/len(os.listdir("img_train"))/0.95)+"%")
        label_dir = line.split()[1]
        image_label = cv2.imread(label_dir, cv2.IMREAD_GRAYSCALE)
        calc(image_label)
        area_calc(image_label)
        image_num += 1


for cls in range(NUM_CLASSES):
    area[cls] = area[cls] / (image_num * 256.0 * 256.0)
area[255] = area[255] / (image_num * 256.0 * 256.0)
print(area)
print(area_proportion)

当前已读取0个样本，进度为 0.0%
当前已读取5000个样本，进度为 7.8060600004995875%
当前已读取10000个样本，进度为 15.612120000999175%
当前已读取15000个样本，进度为 23.418180001498765%
当前已读取20000个样本，进度为 31.22424000199835%
当前已读取25000个样本，进度为 39.030300002497945%
当前已读取30000个样本，进度为 46.83636000299753%
当前已读取35000个样本，进度为 54.642420003497115%
当前已读取40000个样本，进度为 62.4484800039967%
当前已读取45000个样本，进度为 70.25454000449629%
当前已读取50000个样本，进度为 78.06060000499589%
当前已读取55000个样本，进度为 85.86666000549546%
当前已读取60000个样本，进度为 93.67272000599506%
{0: 0.05662774723285185, 1: 0.07080380284899565, 2: 0.03490974336102769, 3: 0.03852830397623599, 255: 0.7991304025808889}
{0: {0: 45770, 1: 1325, 2: 16303, 3: 655}, 1: {0: 38196, 1: 1823, 2: 24034, 3: 0}, 2: {0: 44354, 1: 2050, 2: 17467, 3: 182}, 3: {0: 48485, 1: 1731, 2: 13621, 3: 216}}


### （4）对已划分的数据集进行重采样
**在训练过程中发现第三类mIoU明显低于其他类别，并且发现第一类和第三类非常相似，考虑对第三类进行重采样，并进行额外的数据增强。**

In [4]:
import cv2
import numpy as np
import os
import shutil
os.chdir('/home/aistudio/PaddleSeg/datasets/')

PROB = 0.1  #重采样概率

with open('train_list.txt', 'r') as f:  #把txt写入list中
    lines = f.readlines()

#如果发现类别3存在于img中，那么把该图像的位置在train_list里复制一次
with open('train_list.txt', 'a+') as f:
    index = 0
    count = 0
    samp_count = 0
    for line in lines:
        if index % 5000 == 0:
            print("当前已读取"+str(index)+"个样本，进度为 "+str(100*index/len(lines))+"%")
        label_dir = line.split()[1]
        train_dir = line.split()[0]
        image_label = cv2.imread(label_dir, cv2.IMREAD_GRAYSCALE)
        if 3 in image_label[0]:
            img = cv2.imread(train_dir, cv2.IMREAD_COLOR)
            count += 1
            if np.random.random()<PROB:
                f.write("img_train/T{}.jpg".format("RE"+str(index))+' '+"lab_train/T{}.png".format("RE"+str(index))+"\n")
                rand_rotate = np.random.randint(0,3)
                img_t = cv2.rotate(cv2.bilateralFilter(img, 9, 75, 75), rand_rotate) #随机旋转并且进行双边模糊
                lab_t = cv2.rotate(image_label, rand_rotate)
                cv2.imwrite("img_train/T{}.jpg".format("RE"+str(index)),img_t)
                cv2.imwrite("lab_train/T{}.png".format("RE"+str(index)),lab_t)
                samp_count += 1
        index += 1

print("对含有该类的图像数量为：", count)
print("对含有该类的图像并进行重采样的数量为：", samp_count)

当前已读取0个样本，进度为 0.0%
当前已读取5000个样本，进度为 7.806035626746601%
当前已读取10000个样本，进度为 15.612071253493202%
当前已读取15000个样本，进度为 23.4181068802398%
当前已读取20000个样本，进度为 31.224142506986404%
当前已读取25000个样本，进度为 39.030178133733%
当前已读取30000个样本，进度为 46.8362137604796%
当前已读取35000个样本，进度为 54.6422493872262%
当前已读取40000个样本，进度为 62.44828501397281%
当前已读取45000个样本，进度为 70.2543206407194%
当前已读取50000个样本，进度为 78.060356267466%
当前已读取55000个样本，进度为 85.86639189421261%
当前已读取60000个样本，进度为 93.6724275209592%
对含有该类的图像数量为： 8160
对含有该类的图像并进行重采样的数量为： 819


**第二类和第三类的像素点数量都一样少，但第三类的mIoU更低
猜想可能是数量最多第一类影响了第三类的mIoU**

**若同时对第二类进行重采样，mIoU略微下降**

![](https://ai-studio-static-online.cdn.bcebos.com/29b2de03209e4d8e9efcf654d5f6e31535ee14220425445391199fe40ca378b3)


## 5、模型训练

![](https://ai-studio-static-online.cdn.bcebos.com/d3da1d6ef87945a287e71e8287134bcfe5542dee7139402cb5f602097ba5ad54)


In [5]:
#将work中的配置文件复制到PaddleSeg中
%cd ~
!cp work/Configs/segformer_b2_cityscapes_1024x1024_160k.yml PaddleSeg/configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml
!cp work/Configs/custom_256x256.yml PaddleSeg/configs/_base_/custom_256x256.yml

/home/aistudio


In [6]:
%cd ~/PaddleSeg
! python train.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \
       --save_interval 2000 \
       --use_vdl \
       --log_iters 50 \
       --save_dir output/special \
       --do_eval \
#       --resume_model output/special/iter_10000 \

3094/3094 [==============================] - 156s 51ms/step - batch_cost: 0.0504 - reader cost: 9.2913e-05
2022-11-16 17:41:02 [INFO]	[EVAL] #Images: 3094 mIoU: 0.6218 Acc: 0.7837 Kappa: 0.7004 Dice: 0.7621
2022-11-16 17:41:02 [INFO]	[EVAL] Class IoU: 
[0.5991 0.7741 0.621  0.4929]
2022-11-16 17:41:02 [INFO]	[EVAL] Class Precision: 
[0.7796 0.8502 0.7416 0.6823]
2022-11-16 17:41:02 [INFO]	[EVAL] Class Recall: 
[0.7213 0.8963 0.7925 0.6398]
2022-11-16 17:41:03 [INFO]	[EVAL] The model with the best validation mIoU (0.6226) was saved at iter 202000.
2022-11-16 17:41:17 [INFO]	[TRAIN] epoch: 54, iter: 216050/640000, loss: 0.5425, lr: 0.000041, batch_cost: 0.2749, reader_cost: 0.00014, ips: 58.1933 samples/sec | ETA 32:22:43
2022-11-16 17:41:31 [INFO]	[TRAIN] epoch: 54, iter: 216100/640000, loss: 0.5745, lr: 0.000041, batch_cost: 0.2747, reader_cost: 0.00014, ips: 58.2364 samples/sec | ETA 32:21:03
2022-11-16 17:41:44 [INFO]	[TRAIN] epoch: 54, iter: 216150/640000, loss: 0.5054, lr: 0.000041

![](https://ai-studio-static-online.cdn.bcebos.com/13bf511388ef49a483c81b1bfd7377346765b873a93e462b99764b181553f893)
![](https://ai-studio-static-online.cdn.bcebos.com/78d828460d4c4871bdb542d950630af8b4d73e777fca4dcda4eedaf05d0e11d8)


### **逃离鞍点的方法**
**测试出造成梯度爆炸最低学习率lr1（50iters内loss＞5），将模型学习率调整至0.8*lr1训练直到loss大于原先loss的2倍后恢复正常训练。**

## 6、模型评估

In [8]:
# 模型评估
%cd ~/PaddleSeg
! python val.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \
       --model_path output/special/iter_268000/model.pdparams

/home/aistudio/PaddleSeg
2022-11-16 22:46:48 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 640000
loss:
  coef:
  - 1
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 6.0e-05
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 4
  pretrained: https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b2.tar.gz
  type: SegFormer_B2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: datasets
  mode: train
  num_classes: 4
  train_path: datasets/train_list.txt
  transforms:
  - max_scale_factor: 1.25
    min_scale_factor: 0.75
    scale_step_size: 0.25
    type: ResizeStepScaling
  - prob: 0.075
    type: RandomBlur
  - crop_size:
    - 256
    - 256
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - type: RandomVerticalFlip
  - brightness_range: 0.4
    contrast_range:

![](https://ai-studio-static-online.cdn.bcebos.com/5691eeaecda0442ea37c877469ab242e10d3fe7974ce45ffaa5794afced523f6)


## 7、测试结果

In [11]:
# 测试集预测
%cd ~/PaddleSeg
!python predict.py \
       --config configs/segformer/segformer_b2_cityscapes_1024x1024_160k.yml \
       --model_path output/special/iter_268000/model.pdparams \
       --image_path datasets/img_testA \
       --save_dir result

/home/aistudio/PaddleSeg
2022-11-16 22:50:16 [INFO]	
---------------Config Information---------------
batch_size: 16
iters: 640000
loss:
  coef:
  - 1
  types:
  - coef:
    - 0.8
    - 0.2
    losses:
    - type: CrossEntropyLoss
    - type: LovaszSoftmaxLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 6.0e-05
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 4
  pretrained: https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b2.tar.gz
  type: SegFormer_B2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: datasets
  mode: train
  num_classes: 4
  train_path: datasets/train_list.txt
  transforms:
  - max_scale_factor: 1.25
    min_scale_factor: 0.75
    scale_step_size: 0.25
    type: ResizeStepScaling
  - prob: 0.075
    type: RandomBlur
  - crop_size:
    - 256
    - 256
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - type: RandomVerticalFlip
  - brightness_range: 0.4
    contrast_range:

In [12]:
# 由预测结果生成提交文件
%cd ~/PaddleSeg
!zip -r result.zip result/

  adding: result/added_prediction/919.jpg (deflated 2%)
  adding: result/added_prediction/7649.jpg (deflated 4%)
  adding: result/added_prediction/9624.jpg (deflated 3%)
  adding: result/added_prediction/1288.jpg (deflated 3%)
  adding: result/added_prediction/2786.jpg (deflated 1%)
  adding: result/added_prediction/9598.jpg (deflated 3%)
  adding: result/added_prediction/855.jpg (deflated 3%)
  adding: result/added_prediction/6984.jpg (deflated 3%)
  adding: result/added_prediction/5199.jpg (deflated 2%)
  adding: result/added_prediction/825.jpg (deflated 2%)
  adding: result/added_prediction/3849.jpg (deflated 2%)
  adding: result/added_prediction/8097.jpg (deflated 3%)
  adding: result/added_prediction/7961.jpg (deflated 2%)
  adding: result/added_prediction/170.jpg (deflated 3%)
  adding: result/added_prediction/7328.jpg (deflated 3%)
  adding: result/added_prediction/7870.jpg (deflated 2%)
  adding: result/added_prediction/2279.jpg (deflated 3%)
  adding: result/added_prediction/9

![](https://ai-studio-static-online.cdn.bcebos.com/5bc884da0d514ec3902c21a4dfebffb8d581edd09ceb42219ec5c8ea6f4e04f0)


## 8、基于结果的多模型融合（目前仍存bug）

### **result评分时异常，但可能会涨点**

In [ ]:
!mkdir ~/work/results/merge_result
!mkdir ~/work/results/merge_result/pseudo_color_prediction
!mkdir ~/work/results/merge_result/added_prediction

In [ ]:
import cv2
import numpy as np
import os
import shutil
os.chdir('/home/aistudio/work/results/')

p = 0.1
index = 0

for f in os.listdir("result-segformerb2-65.09/added_prediction"):

    # time_start = time.clock()
    
    if index % 10 == 0:
        print("当前已读取"+str(index)+"个样本，进度为 "+str(100*index/len(os.listdir("result-segformerb2-65.09/added_prediction")))+"%")
    file_name = f.split(".")[0]
    color_map1 = cv2.imread("result-segformerb2-65.09/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    color_map2 = cv2.imread("result-segformerb2-66.24/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    color_map3 = cv2.imread("result-segformerb3/pseudo_color_prediction/{}.png".format(file_name), cv2.IMREAD_COLOR)
    add1 = cv2.imread("result-segformerb2-65.09/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)
    add2 = cv2.imread("result-segformerb2-66.24/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)
    add3 = cv2.imread("result-segformerb3/added_prediction/{}.jpg".format(file_name), cv2.IMREAD_COLOR)

    for i in range(256):
        for j in range(256):
            if (color_map2[i][j] == color_map3[i][j]).all():
                color_map1[i][j] = color_map2[i][j]
                add1[i][j] = add2[i][j]

    cv2.imwrite("merge_result/pseudo_color_prediction/{}.png".format(file_name).format("RE"+str(index)), color_map1)
    cv2.imwrite("merge_result/added_prediction/{}.jpg".format(file_name), add1)

    index += 1

    # time_end = time.clock()
    # tim = time_end - time_start
    # minute = tim % 60
    # sec = tim - 60*minute
    # print("ETA: {}:{}".format(minute, sec))

## 9、总结

![](https://ai-studio-static-online.cdn.bcebos.com/29b829a18a524f288d960b99b87fe027112ebafd530945d3bc5b3346a1cd2ff5)


## 参考项目
https://aistudio.baidu.com/aistudio/projectdetail/4556036